In [1]:
################################################################################
# Program : Taxi_Q_Learning.py
# Description : Open AI GYM의 Taxi 환경의 강화학습 구현 코드 (코랩용) : Q_Learning Taxi
################################################################################

In [2]:
##################################################
# install package
##################################################
!apt-get install -y xvfb x11-utils
!pip install pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 libxtst6 libxxf86dga1 x11-xkb-utils xfonts-base
  xfonts-encodings xfonts-utils xserver-common
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 libxtst6 libxxf86dga1 x11-utils x11-xkb-utils xfonts-base
  xfonts-encodings xfonts-utils xserver-common xvfb
0 upgraded, 12 newly installed, 0 to remove and 23 not upgraded.
Need to get 8,045 kB of archives.
After this operation, 12.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfontenc1 amd64 1:1.1.4-1build3 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxfont2 amd64 1:2.0.5-1build1 [94.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxkbfile1 amd64 1:1.1.0-1build3 [71.8 kB]
Get:4 http://archive.ubuntu.

In [3]:
##################################################
# import package
##################################################
import random
import gym
import numpy as np
import matplotlib.pyplot as plt
from joblib import dump, load

In [4]:
##################################################
# Define Class and Function
##################################################
# Q-Table Update
def table_update(state, action, reward, next_state, alpha, gamma):
    best_qa = max([q_table[(next_state, a)] for a in range(env.action_space.n)])
    q_table[(state, action)] += alpha * (reward + gamma * best_qa - q_table[(state, action)])

#엡실론그리디
def epsilon_greedy(state, epsilon):
    if random.uniform(0,1) < epsilon:
        return env.action_space.sample()
    else:
        return max(list(range(env.action_space.n)), key = lambda x: q_table[(state,x)])


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
##################################################
# Hyperparameters
##################################################
alpha = 0.4
gamma = 0.999
epsilon = 0.017

In [6]:
##################################################
# Set environment
##################################################
env = gym.make('Taxi-v3')

# Q-Table 생성
q_table = {}
for s in range(env.observation_space.n):
    for a in range(env.action_space.n):
        q_table[(s,a)] = 0.0

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [7]:
##################################################
# Train
##################################################
num_episodes = 10000
QL_train_Total_Rewards = []

for episode in range(num_episodes):
    total_reward = 0
    state = env.reset()

    while True:
        env.render()

        # 엡실론 그리디로 행동 선정
        action = epsilon_greedy(state, epsilon)

        # 행동에 따른 다음 상태, 보상, 종료여부 정의
        next_state, reward, done, _ = env.step(action)

        # Q-Table 업데이트
        table_update(state, action, reward, next_state, alpha, gamma)

        state = next_state
        total_reward += reward

        if done:
            break

    QL_train_Total_Rewards.append(total_reward)
    print(f"Episode {episode + 1}: Total Reward = {total_reward}")

env.close()


/usr/local/lib/python3.10/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


Episode 1: Total Reward = -551
Episode 2: Total Reward = -551
Episode 3: Total Reward = -317


KeyboardInterrupt: ignored

In [ ]:
##################################################
# Save
##################################################
# 현재까지 최적화된 Q-Table 저장
dump(q_table, '/content/drive/MyDrive/RL/Taxi_q_table.joblib')
# 에피소드별 보상 저장
dump(QL_train_Total_Rewards, '/content/drive/MyDrive/RL/Taxi_QL_train_Total_Rewards.joblib')